In [1]:
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display
from numpy import NaN

Most of this workbook is code you can also find in app.py, this worbook was mainly a testing ground for new functions and creating the necessary files I need

In [7]:
x = pd.read_csv("ISO 2 to 3.csv")
dx = pd.DataFrame(x, columns=["Country","Alpha-2 code","Alpha-3 code"])
dictISO = {}
for x in dx.iterrows():
    dictISO.update({x[1]["Alpha-2 code"].replace("\"", "").replace(" ", ""): x[1]["Country"].replace("\"", "")})
    
f = pd.read_csv("GDP data.csv", encoding= "latin")
df = pd.DataFrame(f)
dict2020 = {}
dict2021 = {}
dict2022 = {}
for i in df.iterrows():
    dict2020[i[1]["Country"]] =  i[1]["2020"]
    dict2021[i[1]["Country"]] =  i[1]["2021"]
    dict2022[i[1]["Country"]] =  i[1]["2020.1"]

First I set up the sql file and then I add the csv data to it in chunks of 20k

In [60]:
disk_engine = create_engine('sqlite:///311_8M.db')

In [101]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('C:\\Users\\amisa\\Documents\\Python\\Final_Project.git\\2022_All_Countries_Region_Mobility_Report.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['date'] = pd.to_datetime(df['date']) # Convert to datetimes
    df["retail_and_recreation_percent_change_from_baseline"]    = (df["retail_and_recreation_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["grocery_and_pharmacy_percent_change_from_baseline"]     = (df["grocery_and_pharmacy_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["parks_percent_change_from_baseline"]                    = (df["parks_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["transit_stations_percent_change_from_baseline"]         = (df["transit_stations_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["workplaces_percent_change_from_baseline"]               = (df["workplaces_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["residential_percent_change_from_baseline"]              = (df["residential_percent_change_from_baseline"].fillna(method= "ffill")) + 100
    df["Country"]                                               = df["country_region_code"].map(dictISO)
    df.index += index_start

    # Remove the un-interesting columns
    columns = ["Country","date","retail_and_recreation_percent_change_from_baseline","grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline","transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline","residential_percent_change_from_baseline"]

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

First getting all the country names in the SQL db and then turning those into a list

In [102]:
names = pd.read_sql_query("SELECT DISTINCT(Country) from data", disk_engine)

In [103]:
contained_names = []
for x in names.iterrows():
    contained_names.append(x[1]["Country"])

I sample the fist thousand entiries for each country, and get the average for each day, but even after that I realized there is just too much data to make this run smoothly, so instead of take the average for every day, I decided to take the average per week, leaving me with roughly 52 data points per country.

In [104]:
Samples = pd.DataFrame()
for x in contained_names:
    i = pd.read_sql_query(f"SELECT Country,date, AVG(retail_and_recreation_percent_change_from_baseline),AVG(grocery_and_pharmacy_percent_change_from_baseline),AVG(parks_percent_change_from_baseline),AVG(transit_stations_percent_change_from_baseline),AVG(workplaces_percent_change_from_baseline),AVG(residential_percent_change_from_baseline) from data where Country == \"{x}\" GROUP BY date LIMIT 1000", disk_engine)
    Samples = pd.concat([Samples,i])

In [13]:

Samples['date'] = pd.to_datetime(Samples['date'], )
Final_df = Samples.groupby(["Country",pd.Grouper(freq="w",key = "date")]).mean()["AVG(retail_and_recreation_percent_change_from_baseline)"].reset_index()
Final_df["GDP 2020"]= Final_df["Country"].map(dict2020)
Final_df["GDP 2021"]= Final_df["Country"].map(dict2021)
Final_df["GDP 2022"]= Final_df["Country"].map(dict2022)

fig = px.scatter_geo(Final_df, locations="Country", 
                     locationmode= "country names",
                     hover_name="Country", 
                     size="AVG(retail_and_recreation_percent_change_from_baseline)",
                     color = f"GDP {2020}",
                     animation_frame=Final_df.date.astype(str),
                     projection="natural earth")
fig.show()
Final_df.to_parquet("Averages all categories")


,Country,date,AVG(retail_and_recreation_percent_change_from_baseline),GDP 2020,GDP 2021,GDP 2022
0,Afghanistan,2020-02-16,95.250000,611.27,NaN,NaN
1,Afghanistan,2020-02-23,99.785714,611.27,NaN,NaN
2,Afghanistan,2020-03-01,99.285714,611.27,NaN,NaN
3,Afghanistan,2020-03-08,101.714286,611.27,NaN,NaN
4,Afghanistan,2020-03-15,99.000000,611.27,NaN,NaN
...,...,...,...,...,...,...
18697,Zimbabwe,2022-09-18,213.959868,1526.16,2121.57,2420.22
18698,Zimbabwe,2022-09-25,210.645519,1526.16,2121.57,2420.22
18699,Zimbabwe,2022-10-02,216.618222,1526.16,2121.57,2420.22
18700,Zimbabwe,2022-10-09,216.992101,1526.16,2121.57,2420.22
